In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create scribbles

Load the ground truths as batch

In [19]:
img_nums = [1792] #[n for n in range(0, 4900, 500)] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://huggingface.co/api/datasets/EduardoPacheco/FoodSeg103

Check and filter for resolution

In [ ]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

In [ ]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Define scribbles parameters

In [ ]:
# Which scribbles to create
bins = [0.05]#[0.05, 0.3, 1.5]
suff = ["w3"] #standing for width 3
sq_scaling = 100
scribble_width = 3
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/run02"

save_res = False
show_res = True
print_steps = True

Loop and create scribbles

In [ ]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        for s in suff:
            np.random.seed(scribbles_seed)
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, sq_scaling=sq_scaling, mode="all",
                                                            save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps, scribble_width=scribble_width)
            percentages[bin].append(perc_labelled)
            print(f"Annotation percentage: {perc_labelled:2.2f}%\n")

IMG 1792: bin 0.05, suff w3
CLASS 1:
sk_max_pix: 12.33, sq_size: 44, sk_pix_range: (12, 88)
   prim_sk_squares pix: 12 = 0.02%
Adjusting square size and range to 22 (11, 88)
Adjusting square size and range to 11 (5, 88)
   sec_sk_squares pix: 11 = 0.01%
lines_max_pix: 12.33, line_pix_range: (12, 88)
   lines pix: 12 = 0.02%
TOTAL pix: 35 = 0.05%
CLASS 90:
sk_max_pix: 3.11, sq_size: 44, sk_pix_range: (3, 88)
Adjusting square size and range to 22 (3, 88)
Adjusting square size and range to 11 (3, 88)
Adjusting square size and range to 5 (2, 88)
Adjusting square size and range to 2 (1, 88)
   prim_sk_squares pix: 3 = 0.02%
Adjusting square size and range to 22 (3, 88)
Adjusting square size and range to 11 (3, 88)
Adjusting square size and range to 5 (2, 88)
Adjusting square size and range to 2 (1, 88)
   sec_sk_squares pix: 3 = 0.02%
lines_max_pix: 3.11, line_pix_range: (3, 88)
Adjusting line range to (1, 176)
Adjusting distance to edge to 14
Adjusting distance to edge to 21
   lines pix: 

In [11]:
np.sum(scribbles>0) / np.prod(scribbles.shape)

0.009561643276762403

Quick report of labelling percentages:

In [7]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 0.3: mean = 0.9562, std = 0.0000, range = 0.9562 - 0.9562


Show the last created scribble with its ground truth, and all ground truths (optionally)

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")